In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("online_retail.csv")

In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [9]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [11]:
df = df[df['Quantity'] > 0]

In [13]:
df = df.dropna(subset=['CustomerID'])

In [15]:
df['Revenue'] = df['Quantity'] * df['UnitPrice']

In [17]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [19]:
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month

In [21]:
top_customers = df.groupby("CustomerID").agg(
    total_revenue = ("Revenue", "sum"),
    total_quantity = ("Quantity", "sum"),
    avg_unit_price = ("UnitPrice", "mean"),
    invoice_count = ("InvoiceNo", "count"),
)

top_customers = top_customers.sort_values("total_revenue", ascending=False).head(5)
top_customers

,total_revenue,total_quantity,avg_unit_price,invoice_count
CustomerID,,,,
14646.0,280206.02,197491,2.488505,2080
18102.0,259657.30,64124,4.503295,431
17450.0,194550.79,69993,3.392641,337
16446.0,168472.50,80997,1.660000,3
14911.0,143825.06,80515,4.612598,5677


In [23]:
country_month = df.groupby(["Country", "Year", "Month"]).agg(
    monthly_revenue = ("Revenue", "sum"),
    transaction_count = ("InvoiceNo", "nunique"),
    total_quantity = ("Quantity", "sum")
).reset_index()

country_month.head()

,Country,Year,Month,monthly_revenue,transaction_count,total_quantity
0,Australia,2010,12,1032.85,3,467
1,Australia,2011,1,9017.71,9,5644
2,Australia,2011,2,14695.42,5,8680
3,Australia,2011,3,17223.99,3,10439
4,Australia,2011,4,771.60,2,225


In [25]:
pivot_rev = pd.pivot_table(
    df,
    values="Revenue",
    index="Month",
    columns="Country",
    aggfunc="sum",
    fill_value=0
)

pivot_rev

Country,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel Islands,Cyprus,Czech Republic,Denmark,...,RSA,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Unspecified
Month,,,,,,,,,,,,,,,,,,,,,
1,9017.71,0.00,0.0,1200.20,0.0,0.00,675.58,547.50,0.00,0.00,...,0.00,0.00,2053.07,10086.09,3486.32,4231.23,0.00,0.00,442190.060,0.00
2,14695.42,518.36,0.0,2181.07,0.0,0.00,1784.71,4334.24,549.26,399.22,...,0.00,145.92,0.00,2114.50,509.50,2654.92,0.00,889.24,355655.630,0.00
3,17223.99,1708.12,0.0,3351.98,0.0,140.54,3509.33,938.39,0.00,3978.99,...,0.00,0.00,0.00,5363.15,4720.58,1870.23,0.00,0.00,467198.590,0.00
4,771.60,680.78,0.0,1989.48,1143.6,0.00,293.00,0.00,0.00,0.00,...,0.00,0.00,8209.58,1785.65,809.10,2076.94,383.95,0.00,409559.141,299.10
5,13638.41,1249.43,548.4,2732.40,0.0,534.24,1207.24,0.00,0.00,515.70,...,0.00,0.00,0.00,3257.60,2681.98,3610.01,0.00,0.00,551568.820,852.68
6,25187.77,0.00,0.0,4274.82,0.0,1171.46,2060.03,1109.32,0.00,3261.15,...,0.00,0.00,0.00,3333.21,832.00,7904.15,0.00,0.00,524915.480,185.78
7,4964.38,1191.95,0.0,2475.57,0.0,1768.58,0.00,0.00,0.00,376.24,...,0.00,0.00,3949.32,7624.92,6802.09,3762.65,0.00,0.00,485612.251,798.48
8,22489.20,1516.08,0.0,3554.02,0.0,51.56,4896.78,0.00,0.00,213.15,...,0.00,0.00,0.00,3346.91,1400.86,4969.89,0.00,0.00,498453.320,531.03
9,5106.73,0.00,0.0,4208.02,0.0,0.00,1323.75,196.35,0.00,4570.16,...,0.00,0.00,0.00,5189.24,4575.21,8284.86,0.00,1013.04,796780.272,0.00


In [27]:
df[['Revenue', 'Quantity', 'UnitPrice']].describe()

,Revenue,Quantity,UnitPrice
count,397924.000000,397924.000000,397924.000000
mean,22.394749,13.021823,3.116174
std,309.055588,180.420210,22.096788
min,0.000000,1.000000,0.000000
25%,4.680000,2.000000,1.250000
50%,11.800000,6.000000,1.950000
75%,19.800000,12.000000,3.750000
max,168469.600000,80995.000000,8142.750000


In [29]:
aov = (
    df.groupby("CustomerID")
      .agg(total_revenue=("Revenue","sum"), invoices=("InvoiceNo","nunique"))
)

aov['AOV'] = aov['total_revenue'] / aov['invoices']
aov.describe()

,total_revenue,invoices,AOV
count,4339.000000,4339.000000,4339.000000
mean,2053.793018,4.271952,419.047702
std,8988.248381,7.705493,1796.320814
min,0.000000,1.000000,0.000000
25%,307.245000,1.000000,178.505000
50%,674.450000,2.000000,293.690000
75%,1661.640000,5.000000,430.022500
max,280206.020000,210.000000,84236.250000


In [31]:
country_revenue = (
    df.groupby("Country")
      .agg(
          total_revenue=("Revenue","sum"),
          avg_invoice_revenue=("Revenue","mean"),
          avg_unit_price=("UnitPrice","mean"),
          avg_quantity=("Quantity","mean")
      )
      .sort_values("total_revenue", ascending=False)
)

top5_countries = country_revenue.head(5)
top5_countries

,total_revenue,avg_invoice_revenue,avg_unit_price,avg_quantity
Country,,,,
United Kingdom,7308391.554,20.625073,2.963793,12.048913
Netherlands,285446.340,120.798282,2.643982,85.034702
EIRE,265545.900,36.687745,4.445880,19.414894
Germany,228867.140,25.311562,3.708487,13.189892
France,209024.050,25.056827,4.415253,13.362743


In [35]:
df[['Quantity', 'Revenue']].corr()

,Quantity,Revenue
Quantity,1.000000,0.908812
Revenue,0.908812,1.000000


In [37]:
df.to_csv("MaryamShahid_OnlineRetail_Task5.csv", index=False)
print("Cleaned dataset saved as MaryamShahid_OnlineRetail_Task5.csv")

Cleaned dataset saved as MaryamShahid_OnlineRetail_Task5.csv
